In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from time import perf_counter
from itertools import combinations, product
from sortedcontainers import SortedSet
import itertools
import copy

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
def get_distance_matrix(df):
    coords = df[['x', 'y']].to_numpy()
    
    distance_matrix = np.round(squareform(pdist(coords, 'euclidean')))
    np.fill_diagonal(distance_matrix, 0)
    
    return distance_matrix


def random_search(distance_matrix):
    n = len(distance_matrix)
    
    solution = list(range(n))
    np.random.shuffle(solution)
    
    return solution[:(n//2)]


def get_total_cost(solution, distance_matrix, costs):
    assert len(solution) * 2 == len(distance_matrix)
    total_cost = 0
    
    for i in range(len(solution)-1):
        total_cost += distance_matrix[solution[i], solution[i+1]] + costs[solution[i+1]]
        
    total_cost += distance_matrix[solution[-1], solution[0]] + costs[solution[0]]
        
    return total_cost


In [ ]:
def compute_inter_move_delta(solution, distance_matrix, costs, idx, new_node):
    n = len(solution)
    new_solution = solution.copy()
    old_node = solution[idx]

    new = (costs[new_node] +
            distance_matrix[new_solution[idx-1], new_node] +
            distance_matrix[new_node, new_solution[(idx+1)%n]])

    old = (costs[old_node] +
             distance_matrix[new_solution[idx-1], old_node] +
             distance_matrix[old_node, new_solution[(idx+1)%n]])

    delta = new - old
    new_solution[idx] = new_node

    return new_solution, delta



def compute_intra_move_delta(solution, distance_matrix, indices, backward=False):
    n = len(solution)
    i, j = indices

    if i >= j or (i + 1) % n == j % n or (j + 1) % n == i % n:
        return solution.copy(), 0

    start, end = (i, j) if backward else (i + 1, j + 1)

    new = distance_matrix[solution[start - 1], solution[end - 1]] + distance_matrix[solution[start], solution[end % n]]
    old = distance_matrix[solution[start - 1], solution[start]] + distance_matrix[solution[end - 1], solution[end % n]]
    delta = new - old

    new_solution = solution[:start] + solution[start:end][::-1] + solution[end:]
    return new_solution, delta



def steepest_local_search(solution, distance_matrix, costs):
    solution = solution[:]
    n, N = len(solution), len(distance_matrix)
    
    while True:
        best_delta, best_solution = 0, None
        solution_set = set(solution)
        outer_nodes_set = set(range(N)) - solution_set
        
        # inter
        for outer_node, inner_node_idx in product(outer_nodes_set, range(n)):
            new_solution, delta = compute_inter_move_delta(solution, distance_matrix, costs, inner_node_idx, outer_node)
            if delta < best_delta:
                best_delta = delta
                best_solution = new_solution[:]
                
        # intra
        for i, j in combinations(range(n), 2):
            # forward
            new_solution, delta = compute_intra_move_delta(solution, distance_matrix, (i, j), False)
            if delta < best_delta:
                best_delta = delta
                best_solution = new_solution[:]
            # backward
            new_solution, delta = compute_intra_move_delta(solution, distance_matrix, (i, j), True)
            if delta < best_delta:
                best_delta = delta
                best_solution = new_solution[:]
        
        if best_solution is not None:
            solution = best_solution[:]
            continue
        return solution